# Demo for reading and summarizing research articles
Since studies are almost always in PDF format, we need to find a way to read them and get meaningful information out of them. For this purpose, I will be using PyPDF2 to extract the plaintext, and openAI's GPT language model to make sense of the output.


In [1]:
!pip install PyPDF2
!pip install openai

You are using pip version 19.0.3, however version 23.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/35/c3/de7124146c3edbe8fd8163028d9ac998f2fd5dcda9225655f1d4ed684bbc/openai-0.27.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/02/a2cff6306177ae6bc73bc0665065de51dfb3b9db7373e122e2735faf0d97/tqdm-4.65.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/76/ac/a7305707cb852b7e16ff80eaf5692309bde30e2b1100a1fcacdc8f731d97/aiosignal-1.3.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f0/eb/fcb708c7bf5056045e9e98f62b93bd7467eb718b0202e7698eb11d66416c/attrs-23.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/c1/8991e7c5385b897b8c020cdaad718c5b087a6626d1d11a23e1ea87e325a7/async_timeout-4.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/30/bb/bf2944b8b88c65b797acc2c6a2cb0fb817f7364debf0675792e034013858/importlib_metadata-6.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5b/fa/c9e82

You are using pip version 19.0.3, however version 23.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Text Extraction
First, upload a paper in the local directory. Then, use PyPDF2 to iterate and read each page and store each page in a list

In [12]:
from PyPDF2 import PdfReader

#this visitor function helps with cutting off unnecessary header/footer text
def visitor_body(text, cm, tm, font_dict, font_size):
    y = tm[5]
    if y > 10 and y < 780:
        parts.append(text)
        
reader = PdfReader("paper1.pdf")
number_of_pages = len(reader.pages)
pageList = [""] * number_of_pages
for i in range(number_of_pages):
    parts = []
    reader.pages[i].extract_text(visitor_text=visitor_body).lower()
    text_body = "".join(parts)
    # print(text_body)
    pageList[i] = text_body

#print first 100 chars of first page for demonstration
print(pageList[1][0])

2


## Summarize Pages using GPT


In [15]:
import openai

#fill in your openAI apiKey here
openai.api_key = ""

summary = ''
for page in pageList[:2]:
    if page != "":    
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
            {"role": "system", "content": "You are a helpful research assistant."},
            {"role": "user", "content": f"Summarize this academic paper: {page}"},
            ],
        )
        page_summary = response["choices"][0]["message"]["content"]
        summary += page_summary + "\n"

print(summary)




This paper discusses the possible association between ovarian carcinoma and the use of talcum powder. Results from previous studies have been mixed and varied from no significant risk to risks increased by twofold. The study interviewed 450 patients with ovarian carcinoma and 564 population controls in Canada, and found that exposure to talc via sanitary napkins or direct application to the perineum was significantly associated with ovarian carcinoma risk. Duration of exposure also showed a borderline-significant association with risk. The study suggests further investigations on the role of talc in the etiology of ovarian carcinoma are needed.

